In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import xarray as xr

import torch
import gpytorch
from gpytorch.means import ConstantMean
from gpytorch.kernels import ScaleKernel, RBFKernel, InducingPointKernel
from gpytorch.distributions import MultivariateNormal

import tqdm

# subset data 

In [ ]:
byrd_bedmap_corner = byrd_bedmap_points_pixel[(byrd_bedmap_points_pixel["x"] > 525000) & (byrd_bedmap_points_pixel["y"] > - 825000)]

In [ ]:
x_span, y_span = byrd_bedmap_corner.x.max() - byrd_bedmap_corner.x.min(), byrd_bedmap_corner.y.max() - byrd_bedmap_corner.y.min()
x_span/500
y_span/500

# inferene is for 2300
48*48

# Inducing point kernel

In [ ]:
class GPRegressionModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(GPRegressionModel, self).__init__(train_x, train_y, likelihood)
        # CONSTANT MEAN
        self.mean_module = ConstantMean()
        # RBF KERNEL WITH ARD
        self.base_covar_module = ScaleKernel(RBFKernel(ard_num_dims = 2))
        self.covar_module = InducingPointKernel(self.base_covar_module, inducing_points = initial_inducing_points, likelihood = likelihood)

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return MultivariateNormal(mean_x, covar_x)

# Reducing data

In [ ]:
N_size = 5000
N_induce = 500

train_x = torch.tensor(np.array(byrd_bedmap_points_pixel[["x", "y"]]).astype(int)/1000000, dtype = torch.float32)
# generate random order
order = torch.randperm(train_x.size()[0])
# reorder and select first N_size
train_x = train_x[order][:N_size].to(device)

train_y = torch.tensor(np.array(byrd_bedmap_points_pixel[["t_mean"]]), dtype = torch.float32)
train_y = train_y[order][:N_size, 0].to(device)

print(train_x.shape)
print(train_y.shape)

# train_x_small = train_x[:1000].to(device)
# train_y_small = train_x[:1000].to(device)

In [ ]:
initial_inducing_points = train_x[:N_induce]

# Sparse Gaussian Process Model (SGPR) (inducing point kernel)

Scalable kernel approximations.

Inducing point locations should be learned?!

Sparse Gaussian Process Regression (SGPR) (proposed by Titsias, 2009) which approximates kernels using a set of inducing points. This is a general purpose approximation

- scaled RBF kernel as base kernel
- wraped in
- https://docs.gpytorch.ai/en/stable/kernels.html#kernels-for-scalable-gp-regression-methods 
    - Documentation (missing): https://docs.gpytorch.ai/en/stable/kernels.html#gpytorch.kernels.InducingPointKernel
    - Source code: https://docs.gpytorch.ai/en/stable/_modules/gpytorch/kernels/inducing_point_kernel.html#InducingPointKernel

In [ ]:
class GPRegressionModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(GPRegressionModel, self).__init__(train_x, train_y, likelihood)
        # CONSTANT MEAN
        self.mean_module = ConstantMean()
        # RBF KERNEL WITH ARD
        self.base_covar_module = ScaleKernel(RBFKernel(ard_num_dims = 2))
        # INDUCING POINTS
        # Randomly selects 500 first points from the training data (x locations?!)
        # Initialise Inducing points (this is a parameter)
        self.covar_module = InducingPointKernel(self.base_covar_module, inducing_points = initial_inducing_points, likelihood = likelihood)

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return MultivariateNormal(mean_x, covar_x)

In [ ]:
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = GPRegressionModel(train_x, train_y, likelihood)

# Put on cuda
if torch.cuda.is_available():
    model = model.cuda()
    likelihood = likelihood.cuda()

In [ ]:
training_iterations = 1000

# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

def train():
    iterator = tqdm.tqdm(range(training_iterations), desc = "Train")

    for i in iterator:
        # Zero backprop gradients
        optimizer.zero_grad()
        # Get output from model
        output = model(train_x)
        # Calc loss and backprop derivatives
        loss = - mll(output, train_y)
        loss.backward()
        iterator.set_postfix(loss = loss.item())
        optimizer.step()
        torch.cuda.empty_cache()

%time train()

In [ ]:
for name, param in model.named_parameters():
    print(f"Parameter name: {name}")
    print(f"Parameter value: {param}")
    print(f"Requires gradient: {param.requires_grad}")
    print("-----------------------------")

# Why do the inducing points not move

In [ ]:
# XX, YY = np.meshgrid(byrd_bedmachine.x, byrd_bedmachine.y)

fig, ax = plt.subplots(figsize = (10, 10))

# Plot bed topography mesh
# ax.pcolormesh(XX, YY, byrd_bedmachine.thickness_ellipsoid_true, cmap = icethickness_cmap, vmin = 0, vmax = 3500)

# Plot data points
ax.scatter(initial_inducing_points[:, 0].cpu().detach().numpy(), 
           initial_inducing_points[:, 1].cpu().detach().numpy(), 
           c = "green", 
           alpha = 0.5,
           s = 10,
           edgecolors = "green",
           linewidth = 0.15)

ax.scatter(model.covar_module.inducing_points[:, 0].cpu().detach().numpy(), 
           model.covar_module.inducing_points[:, 1].cpu().detach().numpy(), 
           c = "red", 
           s = 10,
           alpha = 0.5,
           edgecolors = "red",
           linewidth = 0.15)

fig.colorbar(mappable = ax.collections[0], ax = ax)
ax.set_aspect('equal')

In [ ]:
# XX, YY = np.meshgrid(byrd_bedmachine.x, byrd_bedmachine.y)

fig, ax = plt.subplots(figsize = (10, 10))

# Plot bed topography mesh
# ax.pcolormesh(XX, YY, byrd_bedmachine.thickness_ellipsoid_true, cmap = icethickness_cmap, vmin = 0, vmax = 3500)

# Plot data points
ax.scatter(initial_inducing_points[:, 0].cpu().detach().numpy(), 
           initial_inducing_points[:, 1].cpu().detach().numpy(), 
           c = "green", 
           s = 10,
           edgecolors = "green",
           linewidth = 0.15)


fig.colorbar(mappable = ax.collections[0], ax = ax)
ax.set_aspect('equal')